In [107]:
# Imports
import pandas as pd
import csv
from datetime import date, timedelta
import folium
import requests
from folium import Marker
from folium.plugins import MarkerCluster
import math
import matplotlib.pyplot as plt
import seaborn as sns
from requests.auth import HTTPBasicAuth
import json
from pandas import json_normalize
import pandas
import numpy as np

In [118]:
#opens json data for vaccine administration for US states and territories
with open('vaccine_admin.json') as f:
    vaccine_admin_json=json.load(f)
print(vaccine_admin_json)
vaccine_admin_json

{'runid': 2206, 'vaccination_data': [{'Date': '2021-10-21', 'Location': 'US', 'ShortName': 'USA', 'LongName': 'United States', 'Census2019': 331996199, 'date_type': 'Report', 'Doses_Distributed': 498702405, 'Doses_Administered': 411010650, 'Dist_Per_100K': 150213, 'Admin_Per_100K': 123800, 'Administered_Moderna': 154219584, 'Administered_Pfizer': 240994970, 'Administered_Janssen': 15371265, 'Administered_Unk_Manuf': 424831, 'Administered_Dose1_Recip': 219624445, 'Administered_Dose1_Pop_Pct': 66.2, 'Administered_Dose2_Pop_Pct': 52.7, 'Administered_Dose1_Recip_18Plus': 204534420, 'Administered_Dose1_Recip_18PlusPop_Pct': 79.2, 'Administered_18Plus': 383317432, 'Admin_Per_100k_18Plus': 148423, 'Distributed_Per_100k_18Plus': 193101, 'Administered_Dose1_Recip_65Plus': 52509902, 'Administered_Dose1_Recip_65PlusPop_Pct': 96.0, 'Administered_65Plus': 105616311, 'Admin_Per_100k_65Plus': 193095, 'Distributed_Per_100k_65Plus': 911762, 'Administered_Dose2_Recip': 174902664, 'Administered_Dose2_Rec

{'runid': 2206,
 'vaccination_data': [{'Date': '2021-10-21',
   'Location': 'US',
   'ShortName': 'USA',
   'LongName': 'United States',
   'Census2019': 331996199,
   'date_type': 'Report',
   'Doses_Distributed': 498702405,
   'Doses_Administered': 411010650,
   'Dist_Per_100K': 150213,
   'Admin_Per_100K': 123800,
   'Administered_Moderna': 154219584,
   'Administered_Pfizer': 240994970,
   'Administered_Janssen': 15371265,
   'Administered_Unk_Manuf': 424831,
   'Administered_Dose1_Recip': 219624445,
   'Administered_Dose1_Pop_Pct': 66.2,
   'Administered_Dose2_Pop_Pct': 52.7,
   'Administered_Dose1_Recip_18Plus': 204534420,
   'Administered_Dose1_Recip_18PlusPop_Pct': 79.2,
   'Administered_18Plus': 383317432,
   'Admin_Per_100k_18Plus': 148423,
   'Distributed_Per_100k_18Plus': 193101,
   'Administered_Dose1_Recip_65Plus': 52509902,
   'Administered_Dose1_Recip_65PlusPop_Pct': 96.0,
   'Administered_65Plus': 105616311,
   'Admin_Per_100k_65Plus': 193095,
   'Distributed_Per_100k_

In [120]:
vaccine_admin_json
vaccine_admin_json['vaccination_data'][1]['LongName']


'Alaska'

In [121]:
#formats the json data into a dataframe
vaccine_admin_df= vaccine_admin_json['vaccination_data']
us_vaccine_admin_df=json_normalize(vaccine_admin_df)

#removes the first row for the general US, unless wanted, I can put it back

us_vaccine_admin_df=us_vaccine_admin_df.iloc[1:,:]

#removes the first column which includes the date
us_vaccine_admin_df=us_vaccine_admin_df.iloc[:,1:]

us_vaccine_admin_df

,Location,ShortName,LongName,Census2019,date_type,Doses_Distributed,Doses_Administered,Dist_Per_100K,Admin_Per_100K,Administered_Moderna,...,Additional_Doses_50Plus,Additional_Doses_50Plus_Vax_Pct,Additional_Doses_18Plus,Additional_Doses_18Plus_Vax_Pct,Additional_Doses_65Plus,Additional_Doses_65Plus_Vax_Pct,Additional_Doses_Moderna,Additional_Doses_Pfizer,Additional_Doses_Janssen,Additional_Doses_Unk_Manuf
1,AK,AKA,Alaska,731545,Report,1125115,847207,153800,115811,323201,...,26282,15.6,36947,10.4,15901,21.2,NaN,NaN,NaN,NaN
2,AL,ALA,Alabama,4903185,Report,6999770,4732829,142760,96526,2049081,...,113302,9.1,134294,6.5,85455,13.2,NaN,NaN,NaN,NaN
3,AR,ARA,Arkansas,3017804,Report,4256950,3161353,141061,104757,1330606,...,75840,10.1,95807,7.2,56253,14.4,NaN,NaN,NaN,NaN
4,AS,ASA,American Samoa,55689,Report,68550,65441,123094,117512,17918,...,8,0.0,16,0.0,1,0.0,NaN,NaN,NaN,NaN
5,AZ,AZA,Arizona,7278717,Report,10410270,8578327,143023,117855,3396918,...,196536,10.1,234139,6.5,152772,14.7,NaN,NaN,NaN,NaN
6,BP2,BP2,Bureau of Prisons,0,Report,262810,235847,0,0,95032,...,485,2.0,1344,1.2,84,2.8,NaN,NaN,NaN,NaN
7,CA,CAA,California,39512223,Report,61020085,52869188,154433,133805,19711065,...,1079731,10.3,1378787,6.2,794340,16.5,NaN,NaN,NaN,NaN
8,CO,COA,Colorado,5758736,Report,8677815,7499427,150690,130227,2883078,...,216933,14.2,278762,8.5,171100,23.7,NaN,NaN,NaN,NaN
9,CT,CTA,Connecticut,3565287,Report,5924555,5265418,166173,147686,1892190,...,144307,11.5,159158,6.8,120643,20.5,NaN,NaN,NaN,NaN
10,DC,DCA,District of Columbia,705749,Report,1276265,1040743,180838,147466,381966,...,15012,9.1,20646,4.9,11016,14.8,NaN,NaN,NaN,NaN


In [115]:
# Population Data
populationData = pd.read_csv('Resources/2019_Census_US_Population_Data_By_State_Lat_Long.csv')

# Get the most recent date for filtering
freshDate = date.today() - timedelta(days=1)
freshDate = date.strftime(freshDate,"%Y%m%d")
freshDate = freshDate[0:4] + "-" + freshDate[4:6] + "-" + freshDate[6:8]



In [116]:
# Vaccination data, for most recent date
vaccinationData = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv')
vaccinationByLocation = vaccinationData.loc[(vaccinationData.date == freshDate)][["location", "people_vaccinated"]]

# Vaccination and population data
vaccinationAndPopulationByLocation = pd.merge(populationData, vaccinationByLocation, left_on='STATE',right_on='location').drop(columns="location")

# Calculate percentage vaccinated by state
vaccinationAndPopulationByLocation["percent_vaccinated"] = vaccinationAndPopulationByLocation["people_vaccinated"] / vaccinationAndPopulationByLocation["POPESTIMATE2019"]

vaccinationAndPopulationByLocation

,STATE,POPESTIMATE2019,lat,long,people_vaccinated,percent_vaccinated
0,Alabama,4903185,32.377716,-86.300568,2637734.0,0.537963
1,Alaska,731545,58.301598,-134.420212,429352.0,0.586911
2,Arizona,7278717,33.448143,-112.096962,4410454.0,0.605938
3,Arkansas,3017804,34.746613,-92.288986,1717255.0,0.569041
4,California,39512223,38.576668,-121.493629,29081847.0,0.736022
5,Colorado,5758736,39.739227,-104.984856,3850188.0,0.668582
6,Connecticut,3565287,41.764046,-72.682198,2781466.0,0.780152
7,Delaware,973764,39.157307,-75.519722,664438.0,0.682340
8,District of Columbia,705749,38.895110,-77.036370,515178.0,0.729973
9,Florida,21477737,30.438118,-84.281296,14669594.0,0.683014
